In [ ]:
# ==== MODULES IMPORT ====
from jupyter_dash import JupyterDash
from dash import ctx
import dash
import dash_leaflet as dl
from dash import dcc, html, Input, Output, State
from dash.exceptions import PreventUpdate
import plotly.express as px
from dash import dash_table as dt
from datetime import datetime
import pandas as pd
import base64
import os

# ==== MONGODB CONNECTION: READ ONLY ====
from animalsCRUD import DBConnection, CRUDObjects
usernameRead = "aacUser"
passwordRead = "aacPass"
readShelter = DBConnection(usernameRead, passwordRead)
readCRUD = CRUDObjects(readShelter.collection)

# ==== LOAD INITIAL DATAFRAME ====
df = pd.DataFrame.from_records(readCRUD.read({}))
if df.empty:
    print("No document read from the collection DB!")

# ==== LOGO SETUP ====
image_filename = 'gs-logo.png'
if os.path.exists(image_filename):
    with open(image_filename, 'rb') as file:
        image_data = file.read()
        encoded_image = base64.b64encode(image_data).decode('utf-8')
else:
    print("Logo image is not available.")
    encoded_image = ''

# ==== FILTER OPTIONS ====
filter_option = [
    'Water Rescue', 'Mountain or Wilderness Rescue',
    'Disaster or Individual Tracking', 'Reset'
]

# ==== DASH APP INITIALIZATION ====
app = JupyterDash("GraziosoApp")

# ==== CLIENT LAYOUT START ====
def clientLayout(df):
    return html.Div([
        html.Div([
            dcc.Input(id='username', type='text', placeholder='Username',
                      style={'marginRight': '10px', 'padding': '6px'}),
            dcc.Input(id='password', type='password', placeholder='Password',
                      style={'marginRight': '10px', 'padding': '6px'}),
            html.Button('Login', id='login-button', n_clicks=0)
        ], style={'display': 'flex', 'alignItems': 'right', 'justifyContent': 'right'}),

        html.Br(), html.Hr(),

        dt.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i} for i in df.columns],
            data=df.to_dict('records'),
            filter_action="native",
            sort_action="native",
            page_action="native",
            page_current=0,
            page_size=10,
            row_selectable="multi",
            selected_rows=[],
            selected_columns=[],
            style_table={'overflowX': 'auto'},
            style_cell={'textAlign': 'left'},
            editable=False
        ),
        html.Br(), html.Hr(),
    ])
# ==== END ====

# ==== ADMIN LAYOUT BGN ====
def adminLayout(df):
    return html.Div([
        html.Div([
            html.Button('Logout', id='logout-button', n_clicks=0)
        ], style={'display': 'flex', 'alignItems': 'right', 'justifyContent': 'right'}),

        html.Br(), html.Hr(),

        # === DATATABLE
        html.Div(style={'display': 'flex'}, children=[
            html.Div([
                html.Button("Add Row", id='add-row-button', n_clicks=0, style={'marginBottom': '10px'}),
                dt.DataTable(
                    id='datatable-id',
                    columns=[{"name": i, "id": i} for i in df.columns],
                    data=df.to_dict('records'),
                    filter_action="native",
                    sort_action="native",
                    page_action="native",
                    page_size=15,
                    row_selectable="multi",
                    row_deletable=True,
                    selected_rows=[],
                    selected_columns=[],
                    editable=True,
                    style_table={'overflowX': 'auto'},
                    style_cell={'textAlign': 'left'}
                )
            ], style={'width': '100%'})
        ]),

        html.Br(), html.Hr(),
    ])
# ==== END ====

# ==== MAIN DASHBOARD LAYOUT ====
app.layout = html.Div([
    html.Div([
        html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '150px', 'height': '150px'})),
        html.Center(html.H3("GRACIOSO SALVARE DASHBOARD")),
        html.Center(html.P(f"Update: {datetime.now().strftime('%m-%d-%Y')}", style={'font-size': '14px', 'color': 'gray'})),
    ], style={'padding': '10px', 'borderBottom': '1px solid #ccc'}),
# ==== FILTERS & SEARCH BAR COMBINED ====
html.Div([
    # LEFT SIDE: QUICK FILTER
    html.Div([
        html.Span("Quick Filter:", style={'marginRight': '10px', 'fontWeight': 'bold'}),
        dcc.RadioItems(
            id='filter-type',
            options=[{'label': i, 'value': i} for i in filter_option],
            value=None,
            labelStyle={'display': 'inline-block', 'marginRight': '15px'},
            style={'display': 'inline-block'}
        )
    ], style={'flex': '1', 'display': 'flex', 'alignItems': 'center'}),

    # RIGHT SIDE: SEARCH BAR + BUTTON
    html.Div([
        dcc.Input(
            id='search-input',
            type='text',
            placeholder='Search by ID, Name, or Type',
            style={'marginRight': '10px', 'padding': '6px', 'width': '250px'}
        ),
        html.Button('Search', id='search-button', n_clicks=0)
    ], style={'flex': '1', 'display': 'flex', 'justifyContent': 'flex-end', 'alignItems': 'center'})
], style={'display': 'flex', 'justifyContent': 'space-between', 'margin': '15px 20px'}),
html.Hr(),

    html.Div(id='main-dashboard', children=clientLayout(df), style={'padding': '30px'}),
    dcc.Store(id='login-status', data=False),
    dcc.Store(id='admin-data-store', data=df.to_dict('records')),

        # === MAP + GRAPH ===
        html.Div(className='row', style={'display': 'flex'}, children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(html.Div(id='map-id', className='col s12 m6'))
        ])
])
# ==== END ==== 

# ==== LOGIN VERIFICATION CALLBACK ====
@app.callback(
    Output('main-dashboard', 'children'),
    Output('login-status', 'data'),
    Input('login-button', 'n_clicks'),
    State('username', 'value'),
    State('password', 'value'),
    prevent_initial_call=True
)
def handle_login(n_clicks, username, password):
    if n_clicks == 0 or username is None or password is None:
        raise PreventUpdate

    if username == 'aacuser' and password == 'aacpass':
        writeShelter = DBConnection(username, password)
        writeCRUD = CRUDObjects(writeShelter.collection)
        df_admin = pd.DataFrame.from_records(writeCRUD.read({}))
        return adminLayout(df_admin), True
    else:
        return html.Div([
            html.H4("Login failed. Try again!", style={'color': 'red'})
        ]), False
# ==== END ====

# ==== CALLBACK FOR TXT BOX BGN ====
@app.callback(
    Output('datatable-id', 'data', allow_duplicate=True),
    Input('search-button', 'n_clicks'),
    State('search-input', 'value'),
    prevent_initial_call=True
)
def search_records(n_clicks, search_value):
    if not search_value:
        raise PreventUpdate

    search_value = search_value.lower()
    filtered_df = df[
        df['animal_id'].str.lower().str.contains(search_value) |
        df['name'].str.lower().str.contains(search_value) |
        df['animal_type'].str.lower().str.contains(search_value)
    ]
    return filtered_df.to_dict('records')
# ==== END ====

# ==== CALLBACK TO ADD ROW ====
@app.callback(
    Output('datatable-id', 'data', allow_duplicate=True),
    Input('add-row-button', 'n_clicks'),
    State('datatable-id', 'data'),
    State('datatable-id', 'columns'),
    prevent_initial_call='initial_duplicate'
)
def add_empty_row(n_clicks, current_data, columns):
     # NEW DIC AND APPEND TO DB
    new_row = {c['id']: "" for c in columns}
    current_data.append(new_row)
    return current_data
# === END === 

# ==== UPDATE DATABASE WHEN A CELL IS MODIFIED ====
@app.callback(
    Input('datatable-id', 'data'),
    State('admin-data-store', 'data'),
    prevent_initial_call=True
)
def auto_update_records(updated_data, original_data):
    writeShelter = DBConnection("aacuser", "aacpass")
    writeCRUD = CRUDObjects(writeShelter.collection)

    updated_df = pd.DataFrame(updated_data)
    original_df = pd.DataFrame(original_data)

    try:
        for i in range(len(updated_df)):
            record = updated_df.iloc[i].to_dict()
            animal_id = record.get('animal_id')

            # ==== ANIMAL ID EXIST UPDATE 
            if animal_id and writeCRUD.read({'animal_id': animal_id}):
                if i < len(original_df) and not updated_df.iloc[i].equals(original_df.iloc[i]):
                    writeCRUD.update({'animal_id': animal_id}, record)
            else:
                # ==== NO ID ADDED NO NEED IT
                writeCRUD.create(record)

        return "Records updated or inserted."
    except Exception as e:
        print(f"ERROR: {e}")
        return "Update failed"
# ==== END ====

# ==== CALLBACK DELETE RECORD FROM DATATABLE BGN ==== 
@app.callback(
    Output('admin-data-store', 'data'),
    Input('datatable-id', 'data'),
    State('admin-data-store', 'data'),
    prevent_initial_call=True
)

# ==== HANDLE RECORD DELETION ====
def handle_deleted_rows(updated_data, original_data):
    updated_ids = {doc['animal_id'] for doc in updated_data}
    original_ids = {doc['animal_id'] for doc in original_data}
    deleted_ids = original_ids - updated_ids

    if deleted_ids:
        writeShelter = DBConnection("aacuser", "aacpass")
        writeCRUD = CRUDObjects(writeShelter.collection)

        for animal_id in deleted_ids:
            writeCRUD.delete({'animal_id': animal_id})

    return updated_data
# ==== END ==== 

# ==== CALLBACK FOR FILTER OPTION BGN ====
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')],
    prevent_initial_call=True
)

# ==== UPDATE DASHBOARD BASED ON FILTERED TYUPE
def update_dashboard(filter_type):
    if filter_type == 'Reset':
        dff = df
    elif filter_type == 'Water Rescue':
        dff = df[df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])]
    elif filter_type == 'Mountain or Wilderness Rescue':
        dff = df[df['breed'].isin(['German Shepherd', 'Alaskan Malamute',
                                   'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])]
    elif filter_type == 'Disaster or Individual Tracking':
        dff = df[df['breed'].isin(['Doberman Pinscher', 'German Shepherd',
                                   'Golden Retriever', 'Bloodhound', 'Rottweiler'])]
    else:
        dff = df

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    return dff.to_dict('records'), columns
# ==== END ====

# ==== CALLBACK FOR MAP AND GRAPH SELECTED COLUMNS ====
@app.callback([
    Output('datatable-id', "style_data_conditional"),
    Output('graph-id', "children"),
    Output('map-id', "children")],
    [Input('datatable-id', "selected_columns"),
     Input('datatable-id', "derived_viewport_data")]
)

# ==== GENERATE MAP AND GRAPH BASE ON SELECT COLUMNS ====
def update_dynamic_objects(selected_columns, viewData):

    # ==== SET COLOR VALUE FOR SELECTED COLUMN
    style = [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

    # ==== CONVERT VIEWPORT TO DF ====
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return style, [], []

    # ==== GRAPH GROUP BY BREED AND COUNT ====
    df_grouped = dff.groupby(['breed']).size().reset_index(name='Count')
    fig = px.pie(df_grouped, values='Count', names='breed', title="Animal Types")
    graph = [dcc.Graph(figure=fig)]

    # ==== MAP COMPONENT WITH MARKERS ====
    map_component = [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ] + [
            dl.Marker(position=[row['location_lat'], row['location_long']], children=[
                dl.Tooltip(row['breed']),
                dl.Popup([html.H1("Animal Name"), html.P(row['name'])])
            ]) for index, row in dff.iterrows()
        ])
    ]

    # ==== RETURN GRAPH AND MAP COMPONENT
    return style, graph, map_component
# ==== END ====


In [ ]:
app.run(jupyter_mode="External")